In [56]:
%matplotlib inline
import os
import pandas as pd
from glob import glob
import numpy as np

In [58]:
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
from path import Path

ModuleNotFoundError: No module named 'path'

In [5]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
genres = 'classical jazz metal pop rock'.split()
for g in genres:
    pathlib.Path(f'dataset/img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./dataset/genres/{g}'):
        songname = f'./dataset/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'dataset/img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

<Figure size 720x720 with 0 Axes>

# Create a spectrogram

In [63]:
def create_spectrogram(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = 'working/train/' + name + '.jpg'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [64]:
def create_spectrogram_test(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('working/test/' + name + '.jpg')
    fig.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

# Extracting features

In [84]:
ids = []
labels = []

In [85]:
file = open('./dataset/data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in genres:
    for filename in os.listdir(f'./dataset/genres/{g}'):
        songname = f'./dataset/genres/{g}/{filename}'
        create_spectrogram(songname,filename)
        
        # extracts the  genre name
        labels.append(filename.split('.')[0])
        ids.append(filename + '.jpg')
        


In [86]:
df = pd.DataFrame({'ID': ids, 'Class': labels})

In [87]:
df.head()

,ID,Class
0,classical.00004.au.jpg,classical
1,classical.00001.au.jpg,classical
2,classical.00002.au.jpg,classical
3,classical.00003.au.jpg,classical
4,classical.00008.au.jpg,classical


# Creating a data generator

In [88]:
from keras_preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1/255, validation_split=.25)

In [89]:
train_generator = datagen.flow_from_dataframe(
    dataframe = df,
    directory="working/train/",
    x_col = "ID",
    y_col = "Class",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64, 64)
)

Found 38 images belonging to 5 classes.


In [93]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
activation_15 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_16 (Activation)   (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 31, 31, 64)        36928     
__________

In [94]:
#Fitting keras model, no test gen for now
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=150
)


Epoch 1/150
1/1 [==============================] - 2s 2s/step - loss: 1.6342 - acc: 0.0938
Epoch 2/150
1/1 [==============================] - 0s 194ms/step - loss: 2.6663 - acc: 0.0000e+00
Epoch 3/150
1/1 [==============================] - 1s 874ms/step - loss: 1.5291 - acc: 0.2500
Epoch 4/150
1/1 [==============================] - 0s 193ms/step - loss: 1.5017 - acc: 0.1667
Epoch 5/150
1/1 [==============================] - 1s 933ms/step - loss: 1.4587 - acc: 0.2500
Epoch 6/150
1/1 [==============================] - 0s 190ms/step - loss: 1.5404 - acc: 0.1667
Epoch 7/150
1/1 [==============================] - 1s 904ms/step - loss: 1.4587 - acc: 0.1562
Epoch 8/150
1/1 [==============================] - 0s 204ms/step - loss: 1.4835 - acc: 0.1667
Epoch 9/150
1/1 [==============================] - 1s 958ms/step - loss: 1.4340 - acc: 0.3125
Epoch 10/150
1/1 [==============================] - 0s 198ms/step - loss: 1.4008 - acc: 0.1667
Epoch 11/150
1/1 [==============================] - 1s 878

Epoch 88/150
1/1 [==============================] - 0s 244ms/step - loss: 0.2149 - acc: 0.8333
Epoch 89/150
1/1 [==============================] - 1s 1s/step - loss: 0.6710 - acc: 0.7188
Epoch 90/150
1/1 [==============================] - 0s 190ms/step - loss: 0.2655 - acc: 1.0000
Epoch 91/150
1/1 [==============================] - 1s 986ms/step - loss: 0.5763 - acc: 0.7500
Epoch 92/150
1/1 [==============================] - 0s 194ms/step - loss: 0.9190 - acc: 0.3333
Epoch 93/150
1/1 [==============================] - 1s 1s/step - loss: 1.1462 - acc: 0.4062
Epoch 94/150
1/1 [==============================] - 0s 291ms/step - loss: 0.5325 - acc: 0.8333
Epoch 95/150
1/1 [==============================] - 1s 996ms/step - loss: 0.6453 - acc: 0.7812
Epoch 96/150
1/1 [==============================] - 0s 246ms/step - loss: 0.3611 - acc: 0.8333
Epoch 97/150
1/1 [==============================] - 1s 948ms/step - loss: 0.5018 - acc: 0.8438
Epoch 98/150
1/1 [==============================] - 0s 1

NameError: name 'valid_generator' is not defined